In [12]:
import numpy as np
import pandas as pd
import timecorr as tc
import hypertools as hyp
import supereeg as se
from matplotlib import pyplot as plt
from matplotlib import animation as ani
import seaborn as sns
import os
import glob as glob
from scipy.io import loadmat as load
import numba
import copy
import nilearn as nl
import nibabel as nib
from nilearn.input_data import NiftiMasker
from scipy.spatial.distance import pdist, cdist, squareform
from collections import deque
from IPython.display import HTML
import matlab.engine as mlab
import matlab

# Core functionality for managing and accessing data
from neurosynth import Dataset
# Analysis tools for meta-analysis, image decoding, and coactivation analysis
from neurosynth import meta, decode, network

%matplotlib inline

In [13]:
pwd

'/Users/lucyowen/repos/timecorr-paper/code/notebooks'

In [14]:
import neurosynth as ns
ns.dataset.download(path='../figs/neurosynth_data/', unpack=True)

8192  [819200.00%16384  [1638400.00%24576  [2457600.00%32768  [3276800.00%40960  [4096000.00%49152  [4915200.00%57344  [5734400.00%65536  [6553600.00%73728  [7372800.00%81920  [8192000.00%90112  [9011200.00%98304  [9830400.00%106496  [10649600.00%114688  [11468800.00%122880  [12288000.00%131072  [13107200.00%139264  [13926400.00%147456  [14745600.00%155648  [15564800.00%163840  [16384000.00%172032  [17203200.00%180224  [18022400.00%188416  [18841600.00%196608  [19660800.00%204800  [20480000.00%212992  [21299200.00%221184  [22118400.00%229376  [22937600.00%237568  [23756800.00%245760  [24576000.00%253952  [25395200.00%262144  [26214400.00%270336  [27033600.00%278528  [27852800.00%286720  [28672000.00%294912  [29491200.00%303104  [30310400.00%311296  [31129600.00%319488  [31948800.00%327680  [32768000.00%335872  [33587200.00%344064  [34406400.00%352256  [35225600.00%360448  [36044800.00%368640  [36864000.00%376832  [37683200.00%385024  [38502400.00%393216  [39321600.00%401408  [40140800.

5054464  [505446400.00%5062656  [506265600.00%5070848  [507084800.00%5079040  [507904000.00%5087232  [508723200.00%5095424  [509542400.00%5103616  [510361600.00%5111808  [511180800.00%5120000  [512000000.00%5128192  [512819200.00%5136384  [513638400.00%5144576  [514457600.00%5152768  [515276800.00%5160960  [516096000.00%5169152  [516915200.00%5177344  [517734400.00%5185536  [518553600.00%5193728  [519372800.00%5201920  [520192000.00%5210112  [521011200.00%5218304  [521830400.00%5226496  [522649600.00%5234688  [523468800.00%5242880  [524288000.00%5251072  [525107200.00%5259264  [525926400.00%5267456  [526745600.00%5275648  [527564800.00%5283840  [528384000.00%5292032  [529203200.00%5300224  [530022400.00%5308416  [530841600.00%5316608  [531660800.00%5324800  [532480000.00%5332992  [533299200.00%5341184  [534118400.00%5349376  [534937600.00%5357568  [535756800.00%5365760  [536576000.00%5373952  [537395200.00%5382144  [538214400.00%5390336  [539033600.00%5398528  [539852800.00%5406720  [5

8118272  [811827200.00%]8126464  [812646400.00%]8134656  [813465600.00%]8142848  [814284800.00%]8151040  [815104000.00%]8159232  [815923200.00%]8167424  [816742400.00%]8175616  [817561600.00%]8183808  [818380800.00%]8192000  [819200000.00%]8200192  [820019200.00%]8208384  [820838400.00%]8216576  [821657600.00%]8224768  [822476800.00%]8232960  [823296000.00%]8241152  [824115200.00%]8249344  [824934400.00%]8257536  [825753600.00%]8265728  [826572800.00%]8273920  [827392000.00%]8282112  [828211200.

9822208  [982220800.00%9830400  [983040000.00%9838592  [983859200.00%9846784  [984678400.00%9854976  [985497600.00%9863168  [986316800.00%9871360  [987136000.00%9879552  [987955200.00%9887744  [988774400.00%9895936  [989593600.00%9904128  [990412800.00%9912320  [991232000.00%9920512  [992051200.00%9928704  [992870400.00%9936896  [993689600.00%9945088  [994508800.00%9953280  [995328000.00%9961472  [996147200.00%9969664  [996966400.00%9977856  [997785600.00%9986048  [998604800.00%9994240  [999424000.00%10002432  [1000243200.00%10010624  [1001062400.00%10018816  [1001881600.00%10027008  [1002700800.00%10035200  [1003520000.00%10043392  [1004339200.00%10051584  [1005158400.00%10059776  [1005977600.00%10067968  [1006796800.00%10076160  [1007616000.00%10084352  [1008435200.00%10092544  [1009254400.00%10100736  [1010073600.00%10108928  [1010892800.00%10117120  [1011712000.00%10125312  [1012531200.00%10133504  [1013350400.00%10141696  [1014169600.00%10149888  [1014988800.00%10158080  [10158080

13082624  [1308262400.00%]13090816  [1309081600.00%]13099008  [1309900800.00%]13107200  [1310720000.00%]13115392  [1311539200.00%]13123584  [1312358400.00%]13131776  [1313177600.00%]13139968  [1313996800.00%]13148160  [1314816000.00%]13156352  [1315635200.00%]13164544  [1316454400.00%]13172736  [1317273600.00%]13180928  [1318092800.00%]13189120  [1318912000.00%]13197312  [1319731200.00%]13205504  [1320550400.00%]13213696  [1321369600.00%]13221888  [1322188800.00%]13230080  [1323008000.00%]

14737408  [1473740800.00%]14745600  [1474560000.00%]14753792  [1475379200.00%]14761984  [1476198400.00%]14770176  [1477017600.00%]14778368  [1477836800.00%]14786560  [1478656000.00%]14794752  [1479475200.00%]14802944  [1480294400.00%]14811136  [1481113600.00%]14819328  [1481932800.00%]14827520  [1482752000.00%]14835712  [1483571200.00%]14843904  [1484390400.00%]14852096  [1485209600.00%]14860288  [1486028800.00%]14868480  [1486848000.00%]14876672  [1487667200.00%]14884864  [1488486400.00%]

16457728  [1645772800.00%]16465920  [1646592000.00%]16474112  [1647411200.00%]16482304  [1648230400.00%]16490496  [1649049600.00%]16498688  [1649868800.00%]16506880  [1650688000.00%]16515072  [1651507200.00%]16523264  [1652326400.00%]16531456  [1653145600.00%]16539648  [1653964800.00%]16547840  [1654784000.00%]16556032  [1655603200.00%]16564224  [1656422400.00%]16572416  [1657241600.00%]16580608  [1658060800.00%]16588800  [1658880000.00%]16596992  [1659699200.00%]16605184  [1660518400.00%]

In [ ]:
dataset = Dataset('../figs/neurosynth_data//database.txt')
